In [ ]:
from keras.models import Sequential, Model
from keras.layers import  Input, LSTM, Bidirectional, Concatenate, Conv1D, Dropout, MaxPooling1D, Flatten, Dense, TimeDistributed
import prettytable
import tensorflow as tf
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score, hamming_loss, accuracy_score
from keras import regularizers, initializers, optimizers, callbacks
from sklearn.model_selection import train_test_split

In [ ]:
# Loading Tensorboard Extension
%load_ext tensorboard

In [ ]:
# Metrics Calculator Function
def evaluate_model(real, predicted):
    accuracy = accuracy_score(real, predicted)
    hamLoss = hamming_loss(real, predicted)
    # element wise correctness
    term_wise_accuracy = np.sum(np.logical_not(
        np.logical_xor(real, predicted)))/real.size

    macro_precision = precision_score(real, predicted, average='macro')
    macro_recall = recall_score(real, predicted, average='macro')
    macro_f1 = f1_score(real, predicted, average='macro')

    micro_precision = precision_score(real, predicted, average='micro')
    micro_recall = recall_score(real, predicted, average='micro')
    micro_f1 = f1_score(real, predicted, average='micro')

    metricTable = prettytable.PrettyTable()
    metricTable.field_names = ["Metric", "Macro Value", "Micro Value"]
    metricTable.add_row(["Hamming Loss", "{0:.3f}".format(hamLoss), ""])
    metricTable.add_row(
        ["Term Wise Accuracy", "{0:.3f}".format(term_wise_accuracy), ""])

    metricTable.add_row(["Accuracy", "{0:.3f}".format(accuracy), ""])
    metricTable.add_row(["Precision", "{0:.3f}".format(
        macro_precision), "{0:.3f}".format(micro_precision)])
    metricTable.add_row(["Recall", "{0:.3f}".format(
        macro_recall), "{0:.3f}".format(micro_recall)])
    metricTable.add_row(
        ["F1-measure", "{0:.3f}".format(macro_f1), "{0:.3f}".format(micro_f1)])

    print(metricTable)

In [ ]:
#Helper Functions
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
  
def initial_boost(epoch):
    if epoch==0: return float(8.0)
    elif epoch==1: return float(4.0)
    elif epoch==2: return float(2.0)
    elif epoch==3: return float(1.5)
    else: return float(1.0)

def step_cyclic(epoch):
    try:
        l_r, decay = 1.0, 0.0001
        if epoch%33==0:multiplier = 10
        else:multiplier = 1
        rate = float(multiplier * l_r * 1/(1 + decay * epoch))
        #print("Epoch",epoch+1,"- learning_rate",rate)
        return rate
    except Exception as e:
        print("Error in lr_schedule:",str(e))
        return float(1.0)


In [ ]:

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33, random_state=42)

STARTING THE MODEL

In [ ]:
inp_layer= Input(shape=(768,1))

In [ ]:
embedding_layer_frozen=TimeDistributed(Dense(units=768, trainable= False))(inp_layer)
embedding_layer_train= TimeDistributed(Dense(units=768, trainable= True))(inp_layer)

In [ ]:
l_lstm_1f =Bidirectional(LSTM(24, return_sequences=True, dropout=0.3, recurrent_dropout=0.0))(embedding_layer_frozen)
l_lstm_1t =Bidirectional(LSTM(24, return_sequences=True, dropout=0.3, recurrent_dropout=0.0))(embedding_layer_train)

l_lstm1 = Concatenate(axis=1)([l_lstm_1f, l_lstm_1t])


In [ ]:
l_lstm1.shape

In [ ]:
l_conv_2 = Conv1D(filters=24, kernel_size=2, activation='relu')(l_lstm1)
l_conv_2 = Dropout(0.3)(l_conv_2)

l_conv_3 = Conv1D(filters=24, kernel_size=3, activation='relu')(l_lstm1)
l_conv_3 = Dropout(0.3)(l_conv_3)

l_conv_5 = Conv1D(filters=24, kernel_size=5, activation='relu')(l_lstm1)
l_conv_5 = Dropout(0.3)(l_conv_5)

l_conv_6 = Conv1D(filters=24, kernel_size=6, kernel_regularizer=regularizers.l2(0.001) ,activation='relu')(l_lstm1)
l_conv_6 = Dropout(0.3)(l_conv_6)

l_conv_8 = Conv1D(filters=24, kernel_size=8, kernel_regularizer=regularizers.l2(0.001) ,activation='relu')(l_lstm1)
l_conv_8 = Dropout(0.3)(l_conv_8)

conv_1 =[l_conv_6, l_conv_5, l_conv_8, l_conv_2, l_conv_3 ]

l_lstm_c = Concatenate(axis =1)(conv_1)

In [ ]:
l_lstm_c.shape

In [ ]:
l_conv_4f = Conv1D(filters= 12, kernel_size=4, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(embedding_layer_frozen)
l_conv_4f = Dropout(0.3)(l_conv_4f)

l_conv_4t = Conv1D(filters= 12, kernel_size=4, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(embedding_layer_train)
l_conv_4t = Dropout(0.3)(l_conv_4t)

l_conv_3f = Conv1D(filters= 12, kernel_size=3, activation='relu')(embedding_layer_frozen)
l_conv_3f = Dropout(0.3)(l_conv_3f)

l_conv_3t = Conv1D(filters= 12, kernel_size=3, activation='relu')(embedding_layer_train)
l_conv_3t = Dropout(0.3)(l_conv_3t)

l_conv_2f = Conv1D(filters= 12, kernel_size=2, activation='relu')(embedding_layer_frozen)
l_conv_2f = Dropout(0.3)(l_conv_2f)

l_conv_2t = Conv1D(filters= 12, kernel_size=2, activation='relu')(embedding_layer_train)
l_conv_2t = Dropout(0.3)(l_conv_2t)

conv_2 = [l_conv_4f, l_conv_4t, l_conv_3f, l_conv_3t, l_conv_2f, l_conv_2t]

l_merge_2 = Concatenate(axis=1)(conv_2)
l_c_lstm = Bidirectional(LSTM(12, return_sequences=True, dropout=0.3, recurrent_dropout=0.0))(l_merge_2)


In [ ]:
l_merge_2.shape

In [ ]:
l_c_lstm.shape

In [ ]:
l_merge = Concatenate(axis=1)([l_lstm_c, l_c_lstm])
l_pool = MaxPooling1D(4)(l_merge)
l_drop = Dropout(0.5)(l_pool)
l_flat = Flatten()(l_drop)
l_dense = Dense(26, activation='relu')(l_flat)
preds= Dense(5, activation='softmax')(l_dense)

In [ ]:
model= Model(inp_layer,preds)
ada_delta = optimizers.Adadelta(lr=0.9, rho= 0.95, epsilon= None, decay =0.002)

lr_metric = get_lr_metric(ada_delta)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Tensorboard Callback** and checkpoint creation

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
callback_tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, batch_size=16, write_grads=True , write_graph=True)
callback_model_checkpoints = callbacks.ModelCheckpoint("checkpoint-{val_loss:.3f}.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=0)
callback_lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [ ]:
model.summary()

In [ ]:
model.save('BalanceNet.h5')

In [ ]:
print("Training Progress:")
model_log = model.fit(X_train, Y_train, validation_data=(X_test, Y_test),
          epochs=50, batch_size=200)

